### Part B


#### Producer 1


In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
import datetime as dt
import os
import pandas as pd

In [ ]:
# change directory (local machine only)
# os.chdir("A2")
# os.chdir("data")

# dirs = os.listdir(os.getcwd())
# print(dirs)

In [ ]:
hostip = "192.168.10.125"

In [ ]:
# reference code taken from FIT3182 applied session week 10
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding="utf-8")
        value_bytes = bytes(value, encoding="utf-8")
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print("Message published successfully. Data: " + str(value))
    except Exception as ex:
        print("Exception in publishing message.")
        print(str(ex))


# reference code taken from FIT3182 applied session week 10
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(
            bootstrap_servers=[f"{hostip}:9092"], api_version=(0, 10)
        )
    except Exception as ex:
        print("Exception while connecting Kafka.")
        print(str(ex))
    finally:
        return _producer


if __name__ == "__main__":
    topic = "climate"
    print("Publishing climate records..")
    producer = connect_kafka_producer()

    # latest date from climate csv (calculated from query)
    latest_date = dt.datetime(2024, 1, 1)

    data = pd.read_csv("climate_streaming.csv")

    # feeding data loop
    while True:
        # randomly feed data (with replacement)
        row = data.sample(n=1).to_dict(orient="records")[0]

        # append producer information to identify the producer
        row["producer_id"] = "climate_producer"

        # append the new date to the row
        row["created_date"] = latest_date.strftime("%Y-%m-%d")

        # call the publish_message function for encoding and sending the message
        publish_message(producer, topic, "parsed", dumps(row))

        # since every 10 seconds = 1 day, new_date = latest date in climate data + 1 day
        latest_date += dt.timedelta(days=1)

        # wait 10 seconds before proceeding
        sleep(10)